In [1]:
import torch
from run_BERT_classifier import preprocessing_for_bert
from run_BERT_classifier import get_input_data
from run_BERT_classifier import BertClassifier

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [13]:
import random
dummy_inputs = preprocessing_for_bert(["bla asdf"])  #tokens und attention mask

In [48]:
# define labels and encode
from sklearn.preprocessing import LabelEncoder
news_classes = ['entertainment', 'politics', 'tech', 'sport', 'business']
num_labels_news = len(news_classes)
le_news = LabelEncoder()
le_news.fit(news_classes)

LabelEncoder()

In [15]:
news_model = BertClassifier(num_labels=num_labels_news)
news_model.load_state_dict(torch.load("models/BERT_classifier_news_data.pt", map_location=device))# load finetuned model

<All keys matched successfully>

In [16]:
# Export to onnx:
torch.onnx.export(news_model, 
                  dummy_inputs, 
                  "news_model.onnx",
                  export_params=True, # store the trained parameter weights inside the model file
                  input_names = ['input_ids', 'input_masks'],   # the model's input names
                  output_names = ['output'], # the model's output names
                 )

In [17]:
# check model:
import onnx

# Load the ONNX model
model = onnx.load("models/news_model.onnx")

# Check that the model is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph torch-jit-export (
  %input_ids[INT64, 1x512]
  %input_masks[INT64, 1x512]
) initializers (
  %bert.embeddings.word_embeddings.weight[FLOAT, 30522x768]
  %bert.embeddings.position_embeddings.weight[FLOAT, 512x768]
  %bert.embeddings.token_type_embeddings.weight[FLOAT, 2x768]
  %bert.embeddings.LayerNorm.weight[FLOAT, 768]
  %bert.embeddings.LayerNorm.bias[FLOAT, 768]
  %bert.encoder.layer.0.attention.self.query.bias[FLOAT, 768]
  %bert.encoder.layer.0.attention.self.key.bias[FLOAT, 768]
  %bert.encoder.layer.0.attention.self.value.bias[FLOAT, 768]
  %bert.encoder.layer.0.attention.output.dense.bias[FLOAT, 768]
  %bert.encoder.layer.0.attention.output.LayerNorm.weight[FLOAT, 768]
  %bert.encoder.layer.0.attention.output.LayerNorm.bias[FLOAT, 768]
  %bert.encoder.layer.0.intermediate.dense.bias[FLOAT, 3072]
  %bert.encoder.layer.0.output.dense.bias[FLOAT, 768]
  %bert.encoder.layer.0.output.LayerNorm.weight[FLOAT, 768]
  %bert.encoder.layer.0.output.LayerNorm.bias[FLOAT, 768]
  %be

## Make inference

In [54]:
label, text = get_input_data("data/bbc-text_test.csv", 445)
print("label: ",label)

label:  tech


In [55]:
import numpy as np
def softmax(x):
    return np.exp(x)/sum(np.exp(x))


In [56]:
import onnxruntime as ort

ort_session = ort.InferenceSession("models/news_model.onnx")
inputs = preprocessing_for_bert([text])
outputs = ort_session.run(
    None,
    {"input_ids": inputs[0].numpy(), "input_masks":inputs[1].numpy()},
)
# print(outputs)  # list with one element: Numpy array of the logits
probs = softmax(outputs[0][0])
pred = np.argmax(probs)
le_news.inverse_transform([pred])[0]

'tech'